In [1]:
import pandas as pd
from glob import glob
import numpy as np
import xml.etree.ElementTree as ET
import os
from typing import List,Union,Final

#### Q1 How much flow each retail brokers have

In [2]:
COLS : Final[str] = ["DataDate","category","venue",
 'orderPct',
 'marketPct',
 'marketableLimitPct',
 'nonMarketableLimitPct',
 'otherPct',
 'netPmtPaidRecvMarketOrdersUsd',
 'netPmtPaidRecvMarketOrdersCph',
 'netPmtPaidRecvMarketableLimitOrdersUsd',
 'netPmtPaidRecvMarketableLimitOrdersCph',
 'netPmtPaidRecvNonMarketableLimitOrdersUsd',
 'netPmtPaidRecvNonMarketableLimitOrdersCph',
 'netPmtPaidRecvOtherOrdersUsd',
 'netPmtPaidRecvOtherOrdersCph']
def readXMLToList(root,datarows:List[List[Union[int,float,str]]]) -> None:
    for monthly in root.findall("rMonthly"):
        for venue in monthly.find("rSP500/rVenues"):
            this_row= []
            this_row.append(int(monthly.find("year").text+monthly.find('mon').text))
            this_row.append("SP500")
            for field in venue:
                if field.tag in {"materialAspects","mic"}:
                    pass
                elif field.tag=="name":
                    this_row.append(field.text)
                else:
                    this_row.append(float(field.text))
            datarows.append(this_row)
        for venue in monthly.find("rOtherStocks/rVenues"):
            this_row= []
            this_row.append(int(monthly.find("year").text+monthly.find('mon').text))
            this_row.append("OtherStocks")
            for field in venue:
                if field.tag in {"materialAspects","mic"}:
                    pass
                elif field.tag=="name":
                    this_row.append(field.text)
                else:
                    this_row.append(float(field.text))
            datarows.append(this_row)
        for venue in monthly.find("rOptions/rVenues"):
            this_row= []
            this_row.append(int(monthly.find("year").text+monthly.find('mon').text))
            this_row.append("Options")
            for field in venue:
                if field.tag in {"materialAspects","mic"}:
                    pass
                elif field.tag=="name":
                    this_row.append(field.text)
                else:
                    this_row.append(float(field.text))
            datarows.append(this_row)
def readByBroker(broker:str,report_path:str,):
    broker_report_path = os.path.join(report_path,broker)
    broker_606_paths = glob(os.path.join(broker_report_path,"*"))
    datarows = []
    for path in broker_606_paths:
        tree = ET.parse(path)
        root = tree.getroot()
        readXMLToList(root,datarows)
    res = pd.DataFrame(data=datarows,columns=COLS)
    res.insert(0,"broker",broker)
    return res

In [3]:
retail_brokers = ["Ameritrade","Chas","Etrade","Fidelity"]
report_path = os.path.abspath(os.path.join(".","downloaded_reports"))

In [4]:
reports606_list = []
for broker in retail_brokers:
    reports606_list.append(readByBroker(broker,report_path))

In [5]:
report606 = pd.concat(reports606_list,axis=0)

In [6]:
report606.sort_values(["broker","DataDate","venue"],inplace=True)
report606.reset_index(inplace=True,drop=True)

In [7]:
report606.groupby("broker").head(1)

,broker,DataDate,category,venue,orderPct,marketPct,marketableLimitPct,nonMarketableLimitPct,otherPct,netPmtPaidRecvMarketOrdersUsd,netPmtPaidRecvMarketOrdersCph,netPmtPaidRecvMarketableLimitOrdersUsd,netPmtPaidRecvMarketableLimitOrdersCph,netPmtPaidRecvNonMarketableLimitOrdersUsd,netPmtPaidRecvNonMarketableLimitOrdersCph,netPmtPaidRecvOtherOrdersUsd,netPmtPaidRecvOtherOrdersCph
0,Ameritrade,202004,SP500,"Citadel Securities, LLC",51.40,51.69,51.48,50.89,51.78,758617.00,12.0000,419590.00,12.0000,1147887.00,33.6800,128688.0,9.11
411,Chas,202004,SP500,"Cboe EDGX Exchange, Inc.",0.44,0.02,0.27,1.10,0.22,-1837.11,-26.9919,-3940.32,-28.9471,10907.24,20.5348,0.0,0.00
1166,Etrade,202010,SP500,"Cboe EDGX Exchange, Inc.",6.33,0.00,0.55,17.14,3.04,0.00,0.0000,-371.74,-7.1393,122814.10,31.5404,0.0,0.00
1896,Fidelity,202001,SP500,Cboe EDGX U.S. Equities Exchange,2.30,0.00,0.03,6.90,0.00,0.00,0.0000,-1.00,-24.2407,97706.00,26.2571,0.0,0.00


In [9]:
report606.columns

Index(['broker', 'DataDate', 'category', 'venue', 'orderPct', 'marketPct',
       'marketableLimitPct', 'nonMarketableLimitPct', 'otherPct',
       'netPmtPaidRecvMarketOrdersUsd', 'netPmtPaidRecvMarketOrdersCph',
       'netPmtPaidRecvMarketableLimitOrdersUsd',
       'netPmtPaidRecvMarketableLimitOrdersCph',
       'netPmtPaidRecvNonMarketableLimitOrdersUsd',
       'netPmtPaidRecvNonMarketableLimitOrdersCph',
       'netPmtPaidRecvOtherOrdersUsd', 'netPmtPaidRecvOtherOrdersCph'],
      dtype='object')

In [10]:
report606 = report606[report606.DataDate>=202101]

In [18]:
monthly_usd_flow = report606.groupby(["broker","DataDate"])[['netPmtPaidRecvMarketOrdersUsd','netPmtPaidRecvMarketableLimitOrdersUsd','netPmtPaidRecvNonMarketableLimitOrdersUsd','netPmtPaidRecvOtherOrdersUsd']].sum()

In [70]:
usd_flow = report606.groupby(["category","broker"])[['netPmtPaidRecvMarketOrdersUsd','netPmtPaidRecvMarketableLimitOrdersUsd','netPmtPaidRecvNonMarketableLimitOrdersUsd','netPmtPaidRecvOtherOrdersUsd']].sum()

In [71]:
usd_flow["usdSum"] = usd_flow.sum(axis=1)

In [72]:
usd_flow

netPmtPaidRecvMarketOrdersUsd  \
category    broker                                      
Options     Ameritrade                   8.128925e+07   
            Chas                         1.351927e+08   
            Etrade                       1.892848e+08   
            Fidelity                     3.698849e+07   
OtherStocks Ameritrade                   5.987950e+07   
            Chas                         9.149248e+07   
            Etrade                       1.340704e+08   
            Fidelity                    -3.197600e+04   
SP500       Ameritrade                   1.268428e+07   
            Chas                         2.556404e+07   
            Etrade                       2.803602e+07   
            Fidelity                    -1.074700e+04   

                        netPmtPaidRecvMarketableLimitOrdersUsd  \
category    broker                                               
Options     Ameritrade                            3.237880e+08   
            Chas                                  1.146639e+08   
            Etrade                                1.562517e+08   
            Fidelity                              2.977143e+07   
OtherStocks Ameritrade                            6.058657e+07   
            Chas                                  2.989516e+07   
            Etrade                                6.165957e+07   
            Fidelity                             -1.422130e+05   
SP500       Ameritrade                            6.186426e+06   
            Chas                                  4.463287e+06   
            Etrade                                5.683118e+06   
            Fidelity                             -2.566200e+04   

                        netPmtPaidRecvNonMarketableLimitOrdersUsd  \
category    broker                                                  
Options     Ameritrade                               2.907176e+08   
            Chas                                     1.616441e+08   
            Etrade                                   1.712239e+08   
            Fidelity                                 1.473838e+08   
OtherStocks Ameritrade                               7.641021e+07   
            Chas                                     8.238255e+07   
            Etrade                                   6.665612e+07   
            Fidelity                                 7.468984e+07   
SP500       Ameritrade                               1.791310e+07   
            Chas                                     2.122931e+07   
            Etrade                                   1.284473e+07   
            Fidelity                                 1.740336e+07   

                        netPmtPaidRecvOtherOrdersUsd        usdSum  
category    broker                                                  
Options     Ameritrade                  1.784988e+08  8.742936e+08  
            Chas                        1.017437e+08  5.132445e+08  
            Etrade                      1.511212e+08  6.678816e+08  
            Fidelity                    3.454680e+07  2.486905e+08  
OtherStocks Ameritrade                  1.818636e+07  2.150626e+08  
            Chas                        1.249772e+07  2.162679e+08  
            Etrade                      2.221758e+07  2.846037e+08  
            Fidelity                   -2.881900e+04  7.448683e+07  
SP500       Ameritrade                  2.353472e+06  3.913727e+07  
            Chas                        3.818600e+06  5.507524e+07  
            Etrade                      6.554766e+06  5.311863e+07  
            Fidelity                   -1.371200e+04  1.735324e+07

In [54]:
tmpA = report606[["netPmtPaidRecvMarketOrdersCph",'netPmtPaidRecvMarketableLimitOrdersCph','netPmtPaidRecvNonMarketableLimitOrdersCph','netPmtPaidRecvOtherOrdersCph']].to_numpy()

In [55]:
tmpB = report606[['marketPct','marketableLimitPct', 'nonMarketableLimitPct', 'otherPct']].to_numpy()

In [56]:
report606[["MarketOrdersCph",'MarketableLimitOrdersCph','NonMarketableLimitOrdersCph','OtherOrdersCph']] = tmpA*tmpB
report606[["MarketOrdersCph",'MarketableLimitOrdersCph','NonMarketableLimitOrdersCph','OtherOrdersCph']] = report606.groupby(["broker","DataDate","category"])[["MarketOrdersCph",'MarketableLimitOrdersCph','NonMarketableLimitOrdersCph','OtherOrdersCph']].transform("sum")/100

In [73]:
Cph_flow_mean = report606.groupby(["category","broker"])[["MarketOrdersCph",'MarketableLimitOrdersCph','NonMarketableLimitOrdersCph','OtherOrdersCph']].mean()

In [74]:
Cph_flow_mean

MarketOrdersCph  MarketableLimitOrdersCph  \
category    broker                                                  
Options     Ameritrade        49.336869                 62.443043   
            Chas              53.439592                 54.222308   
            Etrade            46.123808                 46.324924   
            Fidelity          10.249228                 12.445190   
OtherStocks Ameritrade         9.820313                  9.648559   
            Chas               9.606346                  9.288733   
            Etrade            17.521777                 10.410303   
            Fidelity          -0.002683                 -0.273191   
SP500       Ameritrade         9.994422                  9.994108   
            Chas               9.716730                  9.573645   
            Etrade            19.884293                 18.816069   
            Fidelity          -0.001202                 -0.500677   

                        NonMarketableLimitOrdersCph  OtherOrdersCph  
category    broker                                                   
Options     Ameritrade                    54.218330       37.346005  
            Chas                          52.800062       36.317202  
            Etrade                        45.475222       39.366475  
            Fidelity                      42.307167       17.063959  
OtherStocks Ameritrade                    27.494298        7.738013  
            Chas                          27.517229        8.220717  
            Etrade                        21.177984       11.629815  
            Fidelity                      14.393276       -0.016518  
SP500       Ameritrade                    33.684511        8.845383  
            Chas                          31.260885       10.371896  
            Etrade                        28.953321       16.850117  
            Fidelity                      18.800297       -0.001610